In [1]:
import openrouteservice as ors
import pandas as pd
import requests
import warnings
from urllib3.exceptions import InsecureRequestWarning

#Are code is working, but we need to supress website's expired certificate warning:
warnings.simplefilter('ignore', InsecureRequestWarning)

#OpenRouteService API key
api_key = '5b3ce3597851110001cf6248674dd3584987439a83638a6f102fcd5c'
client = ors.Client(key=api_key, requests_kwargs={'verify':False})

#Reading in Dateframes
worker_list = pd.read_excel('PHL Employee List 2024_02_26.xlsx', sheet_name='PHL Employees')
home_addr = pd.read_excel('Employee_Home_Addresses.xlsx')


#Merge PHL employee list with their home addresses.
merged_df = pd.merge(worker_list, home_addr[['Worker', 'Address 1', 'Address 2', 'City', 'State', 'Postal Code']],
                     on='Worker', how='left')
#Since PHL is located in Stillwater, we are not intrested in the drive time of employees located in Stillwater.
merged_df = merged_df[merged_df['City'].str.lower() != 'stillwater'.lower()]

# Extract the first 5 characters from the "Postal Code" column
merged_df['Postal Code'] = merged_df['Postal Code'].str[:5]


#We need to put the addresses together, since they are seperated across multiple fields.
def combine_address(row):
    address_parts = [#row['Address 1'], row['Address 2'], 
                     row['City'], row['State'], row['Postal Code']]
    # Filter out empty values
    non_empty_parts = [part for part in address_parts if pd.notna(part) and part != '']
    return ', '.join(non_empty_parts)

#Apply combine address function
merged_df['Home Address'] = merged_df.apply(combine_address, axis=1)



def geocode_address(address):
    # Use OpenRouteService Geocoding API to convert address to coordinates
    result = client.pelias_search(address)
    try:
        if result['features']:
            # Extract the first result's coordinates
            coordinates = result['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return None
    except requests.exceptions.SSLError:
        print("SSL Certificate verification failed. Skipping geocoding.")
        return None
    
# Geocode the "Home Address" column and create a new column "Home Coords"
merged_df['Home Coords'] = merged_df['Home Address'].apply(geocode_address)

# Display the resulting DataFrame or perform further operations
#merged_df

In [2]:
#Set the geolocation of the Public Health Lab.
PHL_addr = geocode_address('4615 W Lakeview Rd, Stillwater, OK 74075')
#PHL_addr

In [3]:
#Define function to caculate drive time
def calculate_drive_time(home_coords):
    try:
        # Get directions using OpenRouteService API
        route = client.directions(coordinates=[home_coords, PHL_addr], profile='driving-car', format='geojson')
        
        # Extract and return the duration of the route
        drive_time_seconds = route['features'][0]['properties']['segments'][0]['duration']
        drive_time_minutes = drive_time_seconds // 60
        return drive_time_minutes
    except:
        return None
        
    #return route
# Apply the function to each row to create the "Drive_Time" column
merged_df['Drive_Time'] = merged_df['Home Coords'].apply(calculate_drive_time)
#merged_df

In [4]:
selected_columns = ['Worker', 'Employee ID', 'Home Address', 'Home Coords', 'Drive_Time']
final_df = merged_df[selected_columns]
#final_df

In [5]:
# Save the DataFrame to an Excel file
excel_file_path = 'PHL_Drive_Time.xlsx'
final_df.to_excel(excel_file_path, sheet_name='Drive_Time', index=False)